In [ ]:
import pandas as pd
import numpy as np
import os

# Import raw data

In [ ]:
# Path
path = r"C:\Users\TinaM\Desktop\TMB_File\UTS_JUN_2023\AdvDSL_36114\AT2\ADSI_AT2"

In [ ]:
df_raw = pd.read_csv(path + r"\data\raw\beer_reviews.csv")

# Inspect data

In [ ]:
df_raw.head()

In [ ]:
df_raw.shape

In [ ]:
df_raw.info()
# no missing rows

In [ ]:
df_raw.describe()

# Pre-processing_1

## Drop features

In [ ]:
# Drop the columns
features_drop = ['brewery_id','review_time','review_overall','review_profilename','beer_name','beer_beerid']
df = df_raw.drop(features_drop, axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
brewery_name_counts = df['brewery_name'].value_counts().reset_index()

brewery_name_counts
# there are 5741 different brewery_names

In [ ]:
# target classes
beer_style_counts = df['beer_style'].value_counts().reset_index()

beer_style_counts
# there are 104 beer_styles

In [ ]:
"""
# set the sys.path to the project folder
import os
notebook_dir = os.getcwd() #'c:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2\\notebooks'
project_dir = os.path.dirname(notebook_dir) #'c:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2'
import sys
sys.path.append(project_dir) #'c:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2'
"""

In [ ]:
# split out target
from src.data.sets import pop_target
df_cleaned, target = pop_target(df=df, target_col='beer_style')

In [ ]:
df_cleaned_column_list = df_cleaned.columns.tolist()

# Split dataset

In [ ]:
from src.data.sets import split_sets_random
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target=target, test_ratio=0.2, to_numpy=True)

In [ ]:
# size of the split dataset
print(X_train.shape) #(951968, 6)
print(X_val.shape) #(317323, 6)
print(X_test.shape) #(317323, 6)

### save to /data/interim/

In [ ]:
# save dataset
from src.data.sets import save_sets

save_sets(X_train, y_train, X_val, y_val, X_test, y_test, path='../data/interim/')

In [ ]:
# load dataset
from src.data.sets import load_sets

X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/interim/')

In [ ]:
print(X_train.shape) #(951968, 6)
print(X_val.shape) #(317323, 6)
print(X_test.shape) #(317323, 6)
print(y_train.shape) #(951968, )
print(y_val.shape) #(317323, )
print(y_test.shape) #(317323, )

# Pre-processing 2

## Scale features

In [ ]:
# full feature column names list: df_cleaned_column_list

# categorical feature column name:
features_cate_colname = ['brewery_name']

# numberic feature column name:
features_numeric_colname = [col for col in df_cleaned_column_list if col not in features_cate_colname]

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False)

In [ ]:
# save the scaler for later
from joblib import dump
dump(scaler, '../models/scaler.joblib')
dump(encoder, '../models/encoder.joblib')

In [ ]:
"""
# load scaler and encoder
from joblib import load

scaler = load('../models/scaler.joblib')
encoder = load('../models/encoder.joblib')

from sklearn.compose import ColumnTransformer
# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', scaler, features_numeric_colname),
        ('categorical', encoder, features_cate_colname)
    ])

# Apply the transformations on the training data
X_train_df = pd.DataFrame(X_train, columns=df_cleaned.columns)

X_train_transformed = preprocessor.fit_transform(X_train_df)

# Convert the transformed data to a DataFrame
features_train = pd.DataFrame(X_train_transformed, columns=features_numeric_colname + preprocessor.named_transformers_['categorical'].get_feature_names(features_cate_colname))
# Convert to a function
"""

In [ ]:
# Bre
def cust_process_feature( x, full_feature, categorical_feature, numeric_feature ):

    """
    function to preprocess features of the beer dataset
    x = df with features only
    full_feature = list of all feature names
    categorical_feature = list of categorical column name/s
    numeric_feature = list of numeric column name/s
    """

    from joblib import load
    scaler = load('../models/scaler.joblib')
    encoder = load('../models/encoder.joblib')

    from sklearn.compose import ColumnTransformer
    # Create the ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', scaler, categorical_feature),
            ('categorical', encoder, numeric_feature)
        ])
    
    import pandas as pd
    X_df = pd.DataFrame(x, columns= full_feature )


    from sklearn.pipeline import Pipeline
    from sklearn.compose import make_column_transformer
    
    # Define the column names after one-hot encoding
    cat_feature_names = categorical_transformer.get_feature_names_out(input_features=categorical_feature)

    # Fit and transform the data using the preprocessor
    X_preprocessed = preprocessor.fit_transform(X_df)

    # Convert the preprocessed data back to a DataFrame
    X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=list(categorical_feature) + numeric_feature)

    return X_preprocessed_df

In [ ]:
 X_train_preprocessed_df = cust_process_feature(X_train, df_cleaned_column_list,features_cate_colname,features_numeric_colname)

In [ ]:
 X_train_preprocessed_df